In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

### PHD08 데이터 한글 인식기:
1. 데이터 url:http://www.iapr-tc11.org/mediawiki/index.php/KAIST_Scene_Text_Database
1. 압축을 다 풀고 'kaist_dataset'아래에 이미지 데이터와 xml 파일을 두면
1. 각 xml 파일에서 글자하나하나 위치정보를 읽어 원본 이미지에서 글자를 별도의 이미지 파일로 저장하는 작업을 수행한다.
1. 결과물은 char_data 아래에 각 글자명으로 폴더를 만들고 그 아래에 원본파일명.글자명.idx.jpg로 저장
    - char_data/{character}/{img_filename}.{character}.{idx}.jpg
    - character가 .(dot)인 경우에는 {character}값을 dot으로 대체하여 처리
    

In [2]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

from os import listdir, path, makedirs
from os.path import isfile, isdir, join, basename

base_path = 'phd08_output'

def read_charlist(base_path):
    data_filenames = [path.splitext(f)[0] for f in listdir(base_path) if isfile(join(base_path, f)) and f.lower().endswith('.csv')]        
    
    return data_filenames
        
data_filenames = read_charlist(base_path)
data_filenames = data_filenames[:20]

label_idx = {x:idx for idx, x in enumerate(sorted(data_filenames))}

label_idx

{'가': 0,
 '각': 1,
 '간': 2,
 '갇': 3,
 '갈': 4,
 '갉': 5,
 '갊': 6,
 '감': 7,
 '갑': 8,
 '값': 9,
 '갓': 10,
 '갔': 11,
 '강': 12,
 '갖': 13,
 '갗': 14,
 '같': 15,
 '갚': 16,
 '갛': 17,
 '개': 18,
 '객': 19}

In [3]:
import tensorflow as tf

FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string('train_dir', '/tmp/phd08_train',
                           """Directory where to write event logs """
                           """and checkpoint.""")
tf.app.flags.DEFINE_integer('max_steps', 100000,
                            """Number of batches to run.""")
tf.app.flags.DEFINE_boolean('log_device_placement', False,
                            """Whether to log device placement.""")
tf.app.flags.DEFINE_integer('log_frequency', 10,
                            """How often to log results to the console.""")

In [4]:
FLAGS.batch_size = 128

In [5]:
record_defaults = [
        [''], # FO
        [''], # FS
        [0],  # CP
        [0],  # RE
        [0],  # TH
        [0],  # SL
        [0],  # HE
        [0],  # WD
        [''],  # Kor char.
    ] + [[0]] * (56*56) + [['xx', 0]]

record_defaults

[[''],
 [''],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [''],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 [0],
 

In [6]:
def get_inputs():
    # Create a queue that produces the filenames to read.
    filenames = [join(base_path, '.'.join([char, 'csv'])) for char in data_filenames]
    filename_queue = tf.train.string_input_producer(filenames)
    line_reader = tf.TextLineReader()
    # Read a whole file from the queue, the first returned value in the tuple is the
    # filename which we are ignoring.
    _, csv_row = line_reader.read(filename_queue)
    
    '''
        Columns:
        FO(1): Font type (B:바다, D:돋움, G:고딕, H1:한양해서, H2:헤드라인, M:명조, N:나무, S:샘물, Y:엽서)
        FS(1): Font size (0:12, 1:13: 2:14)
        CP(1): The number of copies (0:0, 1:1, 2:2)
        RE(1): Resolution (0:200, 1:240, 2:280)
        TH(1): Threshold (0:140, 1:180, 2:220)
        SL(1): Slope(Rotate) (0:-3deg, 1:0deg, 2:3deg)
        HE(1): Height(pixels)
        WD(1): Width(pixels)
        Korean Character(1) (가, 각, 간, ...)
        Image data(The number of columns = Rows X Cols)
        Label(1): (0:가, 1:각, 2:간 ...)

    '''
    record_defaults = [
        [''], # FO
        [''], # FS
        [0],  # CP
        [0],  # RE
        [0],  # TH
        [0],  # SL
        [0],  # HE
        [0],  # WD
        [''],  # Kor char.
    ] + [[0]] * (56*56) + [[0]]

    columns = tf.decode_csv(csv_row, record_defaults=record_defaults)
    character = columns[8]
    image = tf.reshape(tf.stack(columns[9:-1]), [56, 56, 1])
    image = tf.cast(image, tf.float32)
    image = tf.image.grayscale_to_rgb(image)
    image = tf.image.per_image_standardization(image)
    label = columns[-1]
    
    '''
    To check valid input
    '''
#     with tf.Session() as sess:
#         coord = tf.train.Coordinator()
#         threads = tf.train.start_queue_runners(sess=sess, coord=coord)
#         fig = plt.figure(figsize=(40,250))
#         for i in range(10):
#             char_value, l_value, image_value = sess.run([character, label, image])
#             print(str(char_value))
#             print(l_value)
#             print(image_value)
#             subplot = fig.add_subplot(50,10,i+1)
#             subplot.set_xlabel(l_value)
#             plt.imshow(image_value)
#         plt.show()            
#         coord.request_stop()
#         coord.join(threads)             

#     print('image', image)
#     print('label', label)
    
    num_preprocess_threads = 1
    min_queue_examples = 256

    images, label_batch = tf.train.batch(
        [image, label],
        batch_size=FLAGS.batch_size,
        num_threads=num_preprocess_threads,
        capacity=min_queue_examples + 3 * FLAGS.batch_size)

#     tf.summary.image('images', images)
#     tf.summary.histogram('label_batch', label_batch)
    return images, tf.reshape(label_batch, [FLAGS.batch_size])

get_inputs()

(<tf.Tensor 'batch:0' shape=(128, 56, 56, 3) dtype=float32>,
 <tf.Tensor 'Reshape_1:0' shape=(128,) dtype=int32>)

In [7]:
NUM_CLASSES = len(label_idx)
NUM_CLASSES

20

In [ ]:
import re
import time
from datetime import datetime

TOWER_NAME = 'tower'
NUM_CLASSES = len(label_idx)


def _activation_summary(x):
    """Helper to create summaries for activations.

    Creates a summary that provides a histogram of activations.
    Creates a summary that measures the sparsity of activations.

    Args:
      x: Tensor
    Returns:
      nothing
    """
    # Remove 'tower_[0-9]/' from the name in case this is a multi-GPU training
    # session. This helps the clarity of presentation on tensorboard.
    tensor_name = re.sub('%s_[0-9]*/' % TOWER_NAME, '', x.op.name)
    tf.summary.histogram(tensor_name + '/activations', x)
    tf.summary.scalar(tensor_name + '/sparsity',
                      tf.nn.zero_fraction(x))


def _variable_on_cpu(name, shape, initializer):
    """Helper to create a Variable stored on CPU memory.

    Args:
      name: name of the variable
      shape: list of ints
      initializer: initializer for Variable

    Returns:
      Variable Tensor
    """
    with tf.device('/cpu:0'):
        dtype = tf.float32
        var = tf.get_variable(name, shape, initializer=initializer, dtype=dtype)
    return var


def _variable_with_weight_decay(name, shape, stddev, wd):
    """Helper to create an initialized Variable with weight decay.

    Note that the Variable is initialized with a truncated normal distribution.
    A weight decay is added only if one is specified.

    Args:
      name: name of the variable
      shape: list of ints
      stddev: standard deviation of a truncated Gaussian
      wd: add L2Loss weight decay multiplied by this float. If None, weight
          decay is not added for this Variable.

    Returns:
      Variable Tensor
    """
    dtype = tf.float32
    var = _variable_on_cpu(
        name,
        shape,
        tf.truncated_normal_initializer(stddev=stddev, dtype=dtype))
    if wd is not None:
        weight_decay = tf.multiply(tf.nn.l2_loss(var), wd, name='weight_loss')
        tf.add_to_collection('losses', weight_decay)
    return var


NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN = 50000
NUM_EXAMPLES_PER_EPOCH_FOR_EVAL = 10000

# Constants describing the training process.
MOVING_AVERAGE_DECAY = 0.9999     # The decay to use for the moving average.
NUM_EPOCHS_PER_DECAY = 350.0      # Epochs after which learning rate decays.
LEARNING_RATE_DECAY_FACTOR = 0.1  # Learning rate decay factor.
INITIAL_LEARNING_RATE = 0.1       # Initial learning rate.

def _add_loss_summaries(total_loss):
    """Add summaries for losses in CIFAR-10 model.

    Generates moving average for all losses and associated summaries for
    visualizing the performance of the network.

    Args:
        total_loss: Total loss from loss().
    Returns:
        loss_averages_op: op for generating moving averages of losses.
    """
    # Compute the moving average of all individual losses and the total loss.
    loss_averages = tf.train.ExponentialMovingAverage(0.9, name='avg')
    losses = tf.get_collection('losses')
    loss_averages_op = loss_averages.apply(losses + [total_loss])

    # Attach a scalar summary to all individual losses and the total loss; do the
    # same for the averaged version of the losses.
    for l in losses + [total_loss]:
        # Name each loss as '(raw)' and name the moving average version of the loss
        # as the original loss name.
        tf.summary.scalar(l.op.name + ' (raw)', l)
        tf.summary.scalar(l.op.name, loss_averages.average(l))

    return loss_averages_op

def _train(total_loss, global_step):
    """Train CIFAR-10 model.

    Create an optimizer and apply to all trainable variables. Add moving
    average for all trainable variables.

    Args:
    total_loss: Total loss from loss().
    global_step: Integer Variable counting the number of training steps
      processed.
    Returns:
    train_op: op for training.
    """
    # Variables that affect learning rate.
    num_batches_per_epoch = NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN / FLAGS.batch_size
    decay_steps = int(num_batches_per_epoch * NUM_EPOCHS_PER_DECAY)

    # Decay the learning rate exponentially based on the number of steps.
    lr = tf.train.exponential_decay(INITIAL_LEARNING_RATE,
                                  global_step,
                                  decay_steps,
                                  LEARNING_RATE_DECAY_FACTOR,
                                  staircase=True)
    tf.summary.scalar('learning_rate', lr)

    # Generate moving averages of all losses and associated summaries.
    loss_averages_op = _add_loss_summaries(total_loss)

    # Compute gradients.
    with tf.control_dependencies([loss_averages_op]):
        opt = tf.train.GradientDescentOptimizer(lr)
        grads = opt.compute_gradients(total_loss)

    # Apply gradients.
    apply_gradient_op = opt.apply_gradients(grads, global_step=global_step)

    # Add histograms for trainable variables.
    for var in tf.trainable_variables():
        tf.summary.histogram(var.op.name, var)

    # Add histograms for gradients.
    for grad, var in grads:
        if grad is not None:
            tf.summary.histogram(var.op.name + '/gradients', grad)

    # Track the moving averages of all trainable variables.
    variable_averages = tf.train.ExponentialMovingAverage(
      MOVING_AVERAGE_DECAY, global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())

    with tf.control_dependencies([apply_gradient_op, variables_averages_op]):
        train_op = tf.no_op(name='train')

    return train_op

def inference(images):
    """Build the CIFAR-10 model.

    Args:
      images: Images returned from distorted_inputs() or inputs().

    Returns:
      Logits.
    """
    # We instantiate all variables using tf.get_variable() instead of
    # tf.Variable() in order to share variables across multiple GPU training runs.
    # If we only ran this model on a single GPU, we could simplify this function
    # by replacing all instances of tf.get_variable() with tf.Variable().
    #
    # conv1
    with tf.variable_scope('conv1') as scope:
        kernel = _variable_with_weight_decay('weights',
                                             shape=[3, 3, 3, 64],
                                             stddev=5e-2,
                                             wd=0.0)
        conv = tf.nn.conv2d(images, kernel, [1, 1, 1, 1], padding='SAME')
        biases = _variable_on_cpu('biases', [64], tf.constant_initializer(0.0))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(pre_activation, name=scope.name)
        _activation_summary(conv1)
        
    # conv1-1
    with tf.variable_scope('conv1-1') as scope:
        kernel = _variable_with_weight_decay('weights',
                                             shape=[3, 3, 64, 64],
                                             stddev=5e-2,
                                             wd=0.0)
        conv = tf.nn.conv2d(conv1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = _variable_on_cpu('biases', [64], tf.constant_initializer(0.0))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv1_1 = tf.nn.relu(pre_activation, name=scope.name)
        _activation_summary(conv1_1)

    # conv1-2
    with tf.variable_scope('conv1-2') as scope:
        kernel = _variable_with_weight_decay('weights',
                                             shape=[3, 3, 64, 64],
                                             stddev=5e-2,
                                             wd=0.0)
        conv = tf.nn.conv2d(conv1_1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = _variable_on_cpu('biases', [64], tf.constant_initializer(0.0))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv1_2 = tf.nn.relu(pre_activation, name=scope.name)
        _activation_summary(conv1_2)
    
    
    # pool1
    pool1 = tf.nn.max_pool(conv1_2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                           padding='SAME', name='pool1')
    # norm1
    norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                      name='norm1')

    # conv2
    with tf.variable_scope('conv2') as scope:
        kernel = _variable_with_weight_decay('weights',
                                             shape=[3, 3, 64, 64],
                                             stddev=5e-2,
                                             wd=0.0)
        conv = tf.nn.conv2d(norm1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = _variable_on_cpu('biases', [64], tf.constant_initializer(0.1))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(pre_activation, name=scope.name)
        _activation_summary(conv2)

    # conv2-1
    with tf.variable_scope('conv2-1') as scope:
        kernel = _variable_with_weight_decay('weights',
                                             shape=[3, 3, 64, 64],
                                             stddev=5e-2,
                                             wd=0.0)
        conv = tf.nn.conv2d(conv2, kernel, [1, 1, 1, 1], padding='SAME')
        biases = _variable_on_cpu('biases', [64], tf.constant_initializer(0.1))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv2_1 = tf.nn.relu(pre_activation, name=scope.name)
        _activation_summary(conv2_1)

    # conv2-2
    with tf.variable_scope('conv2-2') as scope:
        kernel = _variable_with_weight_decay('weights',
                                             shape=[3, 3, 64, 64],
                                             stddev=5e-2,
                                             wd=0.0)
        conv = tf.nn.conv2d(conv2_1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = _variable_on_cpu('biases', [64], tf.constant_initializer(0.1))
        pre_activation = tf.nn.bias_add(conv, biases)
        conv2_2 = tf.nn.relu(pre_activation, name=scope.name)
        _activation_summary(conv2_2)
        
    # norm2
    norm2 = tf.nn.lrn(conv2_2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                      name='norm2')
    # pool2
    pool2 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1],
                           strides=[1, 2, 2, 1], padding='SAME', name='pool2')

    # local3
    with tf.variable_scope('local3') as scope:
        # Move everything into depth so we can perform a single matrix multiply.
        reshape = tf.reshape(pool2, [FLAGS.batch_size, -1])
        dim = reshape.get_shape()[1].value
        weights = _variable_with_weight_decay('weights', shape=[dim, 384],
                                              stddev=0.04, wd=0.004)
        biases = _variable_on_cpu('biases', [384], tf.constant_initializer(0.1))
        local3 = tf.nn.relu(tf.matmul(reshape, weights) + biases, name=scope.name)
        _activation_summary(local3)

    # local4
    with tf.variable_scope('local4') as scope:
        weights = _variable_with_weight_decay('weights', shape=[384, 192],
                                              stddev=0.04, wd=0.004)
        biases = _variable_on_cpu('biases', [192], tf.constant_initializer(0.1))
        local4 = tf.nn.relu(tf.matmul(local3, weights) + biases, name=scope.name)
        _activation_summary(local4)
    print('local3', local3)
    print('local4', local4)

    # linear layer(WX + b),
    # We don't apply softmax here because
    # tf.nn.sparse_softmax_cross_entropy_with_logits accepts the unscaled logits
    # and performs the softmax internally for efficiency.
    with tf.variable_scope('softmax_linear') as scope:
        weights = _variable_with_weight_decay('weights', [192, NUM_CLASSES],
                                              stddev=1 / 192.0, wd=0.0)
        biases = _variable_on_cpu('biases', [NUM_CLASSES],
                                  tf.constant_initializer(0.0))
        
        print('weights', weights)
        print('biases', biases)
        softmax_linear = tf.add(tf.matmul(local4, weights), biases, name=scope.name)
        
        print('softmax_linear', softmax_linear)
        _activation_summary(softmax_linear)

    return softmax_linear

def _loss(logits, labels):
    """Add L2Loss to all the trainable variables.

    Add summary for "Loss" and "Loss/avg".
    Args:
      logits: Logits from inference().
      labels: Labels from distorted_inputs or inputs(). 1-D tensor
              of shape [batch_size]

    Returns:
      Loss tensor of type float.
    """
    # Calculate the average cross entropy loss across the batch.
    labels = tf.to_int64(labels)
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=labels, logits=logits, name='cross_entropy_per_example')
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')
    tf.add_to_collection('losses', cross_entropy_mean)

    # The total loss is defined as the cross entropy loss plus all of the weight
    # decay terms (L2 loss).
    return tf.add_n(tf.get_collection('losses'), name='total_loss')


def train():
    """Train CIFAR-10 for a number of steps."""
    with tf.Graph().as_default():
        images, labels = get_inputs()
        
        print('labels.get_shape()', labels.get_shape())
        global_step = tf.contrib.framework.get_or_create_global_step()
        
        # Build a Graph that computes the logits predictions from the
        # inference model.
        logits = inference(images)
        
        print('images, logits, labels', images, logits, labels)

        # Calculate loss.
        loss = _loss(logits, labels)

        # Build a Graph that trains the model with one batch of examples and
        # updates the model parameters.
        train_op = _train(loss, global_step)

        class _LoggerHook(tf.train.SessionRunHook):
            """Logs loss and runtime."""

            def begin(self):
                self._step = -1
                self._start_time = time.time()

            def before_run(self, run_context):
                self._step += 1
                return tf.train.SessionRunArgs(loss)  # Asks for loss value.

            def after_run(self, run_context, run_values):
                if self._step % FLAGS.log_frequency == 0:
                    current_time = time.time()
                    duration = current_time - self._start_time
                    self._start_time = current_time

                    loss_value = run_values.results
                    examples_per_sec = FLAGS.log_frequency * FLAGS.batch_size / duration
                    sec_per_batch = float(duration / FLAGS.log_frequency)

                    format_str = ('%s: step %d, loss = %.2f (%.1f examples/sec; %.3f '
                                  'sec/batch)')
                    print(format_str % (datetime.now(), self._step, loss_value,
                                        examples_per_sec, sec_per_batch))

        with tf.train.MonitoredTrainingSession(
                checkpoint_dir=FLAGS.train_dir,
                hooks=[tf.train.StopAtStepHook(last_step=FLAGS.max_steps),
                       tf.train.NanTensorHook(loss),
                       _LoggerHook()],
                config=tf.ConfigProto(
                    log_device_placement=FLAGS.log_device_placement)) as mon_sess:
            while not mon_sess.should_stop():
                mon_sess.run(train_op)

train()

labels.get_shape() (128,)
local3 Tensor("local3/local3:0", shape=(128, 384), dtype=float32)
local4 Tensor("local4/local4:0", shape=(128, 192), dtype=float32)
weights <tf.Variable 'softmax_linear/weights:0' shape=(192, 20) dtype=float32_ref>
biases <tf.Variable 'softmax_linear/biases:0' shape=(20,) dtype=float32_ref>
softmax_linear Tensor("softmax_linear/softmax_linear:0", shape=(128, 20), dtype=float32)
images, logits, labels Tensor("batch:0", shape=(128, 56, 56, 3), dtype=float32) Tensor("softmax_linear/softmax_linear:0", shape=(128, 20), dtype=float32) Tensor("Reshape_1:0", shape=(128,), dtype=int32)
INFO:tensorflow:Summary name conv1/weight_loss (raw) is illegal; using conv1/weight_loss__raw_ instead.
INFO:tensorflow:Summary name conv1-1/weight_loss (raw) is illegal; using conv1-1/weight_loss__raw_ instead.
INFO:tensorflow:Summary name conv1-2/weight_loss (raw) is illegal; using conv1-2/weight_loss__raw_ instead.
INFO:tensorflow:Summary name conv2/weight_loss (raw) is illegal; using

2017-09-28 19:46:15.002998: step 630, loss = 1180374.25 (37.6 examples/sec; 3.401 sec/batch)
2017-09-28 19:46:48.707596: step 640, loss = 1170970.00 (38.0 examples/sec; 3.370 sec/batch)
2017-09-28 19:47:22.539389: step 650, loss = 1161638.50 (37.8 examples/sec; 3.383 sec/batch)
2017-09-28 19:47:56.216070: step 660, loss = 1152380.12 (38.0 examples/sec; 3.368 sec/batch)
2017-09-28 19:48:29.946240: step 670, loss = 1143196.00 (37.9 examples/sec; 3.373 sec/batch)
2017-09-28 19:49:01.685390: step 680, loss = 1134084.62 (40.3 examples/sec; 3.174 sec/batch)
2017-09-28 19:49:35.167191: step 690, loss = 1125046.25 (38.2 examples/sec; 3.348 sec/batch)
INFO:tensorflow:global_step/sec: 0.298608
2017-09-28 19:50:09.487324: step 700, loss = 1116079.50 (37.3 examples/sec; 3.432 sec/batch)
2017-09-28 19:50:43.356222: step 710, loss = 1107184.25 (37.8 examples/sec; 3.387 sec/batch)
INFO:tensorflow:Saving checkpoints for 713 into /tmp/phd08_train/model.ckpt.
2017-09-28 19:51:20.829785: step 720, loss =

2017-09-28 20:32:00.149524: step 1440, loss = 617365.94 (37.8 examples/sec; 3.384 sec/batch)
2017-09-28 20:32:33.575342: step 1450, loss = 612444.69 (38.3 examples/sec; 3.343 sec/batch)
2017-09-28 20:33:07.342816: step 1460, loss = 607564.12 (37.9 examples/sec; 3.377 sec/batch)
2017-09-28 20:33:41.286833: step 1470, loss = 602723.50 (37.7 examples/sec; 3.394 sec/batch)
2017-09-28 20:34:13.513978: step 1480, loss = 597918.88 (39.7 examples/sec; 3.223 sec/batch)
2017-09-28 20:34:45.848985: step 1490, loss = 593154.50 (39.6 examples/sec; 3.234 sec/batch)
INFO:tensorflow:global_step/sec: 0.295016
2017-09-28 20:35:20.084139: step 1500, loss = 588426.44 (37.4 examples/sec; 3.424 sec/batch)
2017-09-28 20:35:53.922074: step 1510, loss = 583737.50 (37.8 examples/sec; 3.384 sec/batch)
2017-09-28 20:36:27.582086: step 1520, loss = 579084.69 (38.0 examples/sec; 3.366 sec/batch)
2017-09-28 20:37:01.271221: step 1530, loss = 574469.56 (38.0 examples/sec; 3.369 sec/batch)
2017-09-28 20:37:34.823472: 

2017-09-28 21:18:21.554696: step 2260, loss = 320563.44 (37.6 examples/sec; 3.404 sec/batch)
2017-09-28 21:18:55.425818: step 2270, loss = 318008.12 (37.8 examples/sec; 3.387 sec/batch)
2017-09-28 21:19:29.465988: step 2280, loss = 315473.69 (37.6 examples/sec; 3.404 sec/batch)
2017-09-28 21:20:03.273233: step 2290, loss = 312964.56 (37.9 examples/sec; 3.381 sec/batch)
INFO:tensorflow:global_step/sec: 0.293653
2017-09-28 21:20:38.360029: step 2300, loss = 310465.25 (36.5 examples/sec; 3.509 sec/batch)
INFO:tensorflow:Saving checkpoints for 2309 into /tmp/phd08_train/model.ckpt.
2017-09-28 21:21:13.730175: step 2310, loss = 307991.94 (36.2 examples/sec; 3.537 sec/batch)
2017-09-28 21:21:47.515022: step 2320, loss = 305536.31 (37.9 examples/sec; 3.378 sec/batch)
2017-09-28 21:22:20.360491: step 2330, loss = 303101.41 (39.0 examples/sec; 3.285 sec/batch)
2017-09-28 21:22:53.792458: step 2340, loss = 300687.44 (38.3 examples/sec; 3.343 sec/batch)
2017-09-28 21:23:27.784378: step 2350, loss

2017-09-28 22:04:11.850716: step 3070, loss = 167661.73 (40.0 examples/sec; 3.202 sec/batch)
2017-09-28 22:04:44.455217: step 3080, loss = 166326.53 (39.3 examples/sec; 3.260 sec/batch)
2017-09-28 22:05:18.072254: step 3090, loss = 164999.97 (38.1 examples/sec; 3.362 sec/batch)
INFO:tensorflow:global_step/sec: 0.294901
2017-09-28 22:05:51.924323: step 3100, loss = 163684.94 (37.8 examples/sec; 3.385 sec/batch)
2017-09-28 22:06:25.293038: step 3110, loss = 162384.88 (38.4 examples/sec; 3.337 sec/batch)
2017-09-28 22:06:58.820670: step 3120, loss = 161086.36 (38.2 examples/sec; 3.353 sec/batch)
2017-09-28 22:07:32.987057: step 3130, loss = 159804.30 (37.5 examples/sec; 3.417 sec/batch)
2017-09-28 22:08:06.320645: step 3140, loss = 158529.02 (38.4 examples/sec; 3.333 sec/batch)
2017-09-28 22:08:40.204746: step 3150, loss = 157266.61 (37.8 examples/sec; 3.388 sec/batch)
2017-09-28 22:09:14.182322: step 3160, loss = 156012.84 (37.7 examples/sec; 3.398 sec/batch)
2017-09-28 22:09:47.496238: 

...

2017-09-29 11:31:20.511258: step 17150, loss = 4.44 (26.5 examples/sec; 4.834 sec/batch)
2017-09-29 11:32:05.606350: step 17160, loss = 5.07 (28.4 examples/sec; 4.510 sec/batch)
2017-09-29 11:32:46.548521: step 17170, loss = 4.19 (31.3 examples/sec; 4.094 sec/batch)
INFO:tensorflow:Saving checkpoints for 17182 into /tmp/phd08_train/model.ckpt.
2017-09-29 11:33:31.392135: step 17180, loss = 5.23 (28.5 examples/sec; 4.484 sec/batch)
2017-09-29 11:34:11.736399: step 17190, loss = 5.65 (31.7 examples/sec; 4.034 sec/batch)
INFO:tensorflow:global_step/sec: 0.235173
2017-09-29 11:35:00.277066: step 17200, loss = 4.68 (26.4 examples/sec; 4.854 sec/batch)
2017-09-29 11:35:40.928931: step 17210, loss = 5.22 (31.5 examples/sec; 4.065 sec/batch)
2017-09-29 11:36:19.901695: step 17220, loss = 4.30 (32.8 examples/sec; 3.897 sec/batch)
2017-09-29 11:36:57.240695: step 17230, loss = 5.50 (34.3 examples/sec; 3.734 sec/batch)
2017-09-29 11:37:34.444420: step 17240, loss = 5.96 (34.4 examples/sec; 3.720 

2017-09-29 12:31:06.570847: step 17980, loss = 4.35 (24.6 examples/sec; 5.199 sec/batch)
2017-09-29 12:31:59.014567: step 17990, loss = 3.42 (24.4 examples/sec; 5.244 sec/batch)
INFO:tensorflow:global_step/sec: 0.192538
2017-09-29 12:32:51.502051: step 18000, loss = 3.94 (24.4 examples/sec; 5.249 sec/batch)
INFO:tensorflow:Saving checkpoints for 18010 into /tmp/phd08_train/model.ckpt.
2017-09-29 12:33:51.659454: step 18010, loss = 5.07 (21.3 examples/sec; 6.016 sec/batch)
2017-09-29 12:34:44.201043: step 18020, loss = 4.07 (24.4 examples/sec; 5.254 sec/batch)
2017-09-29 12:35:36.319482: step 18030, loss = 4.47 (24.6 examples/sec; 5.212 sec/batch)
2017-09-29 12:36:28.411704: step 18040, loss = 3.52 (24.6 examples/sec; 5.209 sec/batch)
2017-09-29 12:37:19.901206: step 18050, loss = 4.28 (24.9 examples/sec; 5.149 sec/batch)
2017-09-29 12:38:12.275758: step 18060, loss = 5.04 (24.4 examples/sec; 5.237 sec/batch)
2017-09-29 12:39:04.258293: step 18070, loss = 4.04 (24.6 examples/sec; 5.198 

INFO:tensorflow:global_step/sec: 0.183334
2017-09-29 13:44:30.584353: step 18800, loss = 3.57 (19.8 examples/sec; 6.464 sec/batch)
2017-09-29 13:45:24.739300: step 18810, loss = 2.68 (23.6 examples/sec; 5.415 sec/batch)
2017-09-29 13:46:19.884432: step 18820, loss = 3.05 (23.2 examples/sec; 5.515 sec/batch)
2017-09-29 13:47:12.569247: step 18830, loss = 4.04 (24.3 examples/sec; 5.268 sec/batch)
2017-09-29 13:48:04.736419: step 18840, loss = 3.08 (24.5 examples/sec; 5.217 sec/batch)
2017-09-29 13:48:56.587131: step 18850, loss = 3.27 (24.7 examples/sec; 5.185 sec/batch)
2017-09-29 13:49:48.990342: step 18860, loss = 2.43 (24.4 examples/sec; 5.240 sec/batch)
2017-09-29 13:50:42.375351: step 18870, loss = 3.17 (24.0 examples/sec; 5.339 sec/batch)
2017-09-29 13:51:35.057827: step 18880, loss = 4.36 (24.3 examples/sec; 5.268 sec/batch)
2017-09-29 13:52:27.734714: step 18890, loss = 3.35 (24.3 examples/sec; 5.268 sec/batch)
INFO:tensorflow:global_step/sec: 0.187505
2017-09-29 13:53:23.913249

2017-09-29 14:51:00.478855: step 19630, loss = 5.40 (38.0 examples/sec; 3.372 sec/batch)
2017-09-29 14:51:34.252279: step 19640, loss = 5.47 (37.9 examples/sec; 3.377 sec/batch)
2017-09-29 14:52:07.777421: step 19650, loss = 6.49 (38.2 examples/sec; 3.353 sec/batch)
2017-09-29 14:52:41.649675: step 19660, loss = 5.50 (37.8 examples/sec; 3.387 sec/batch)
2017-09-29 14:53:15.439755: step 19670, loss = 6.32 (37.9 examples/sec; 3.379 sec/batch)
2017-09-29 14:53:49.277095: step 19680, loss = 5.34 (37.8 examples/sec; 3.384 sec/batch)
INFO:tensorflow:Saving checkpoints for 19687 into /tmp/phd08_train/model.ckpt.
2017-09-29 14:54:26.716475: step 19690, loss = 5.82 (34.2 examples/sec; 3.744 sec/batch)
INFO:tensorflow:global_step/sec: 0.286925
2017-09-29 14:55:00.809162: step 19700, loss = 7.09 (37.5 examples/sec; 3.409 sec/batch)
2017-09-29 14:55:34.509825: step 19710, loss = 6.25 (38.0 examples/sec; 3.370 sec/batch)
2017-09-29 14:56:08.332469: step 19720, loss = 6.57 (37.8 examples/sec; 3.382 

2017-09-29 15:40:00.760887: step 20470, loss = 5.54 (38.1 examples/sec; 3.355 sec/batch)
2017-09-29 15:40:35.431866: step 20480, loss = 4.54 (36.9 examples/sec; 3.467 sec/batch)
2017-09-29 15:41:07.802121: step 20490, loss = 4.78 (39.5 examples/sec; 3.237 sec/batch)
INFO:tensorflow:global_step/sec: 0.29739
2017-09-29 15:41:41.922390: step 20500, loss = 3.84 (37.5 examples/sec; 3.412 sec/batch)
2017-09-29 15:42:15.599660: step 20510, loss = 3.06 (38.0 examples/sec; 3.368 sec/batch)
2017-09-29 15:42:48.990005: step 20520, loss = 4.07 (38.3 examples/sec; 3.339 sec/batch)
2017-09-29 15:43:22.626579: step 20530, loss = 3.53 (38.1 examples/sec; 3.364 sec/batch)
2017-09-29 15:43:56.364944: step 20540, loss = 4.93 (37.9 examples/sec; 3.374 sec/batch)
INFO:tensorflow:Saving checkpoints for 20548 into /tmp/phd08_train/model.ckpt.
2017-09-29 15:44:33.254040: step 20550, loss = 3.92 (34.7 examples/sec; 3.689 sec/batch)
2017-09-29 15:45:06.754688: step 20560, loss = 4.55 (38.2 examples/sec; 3.350 s